In [ ]:
!nvidia-smi

Sun Nov  7 15:42:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:38tcmalloc: large alloc 1147494400 bytes == 0x563b62dae000 @  0x7f5141db1615 0x563b298a94cc 0x563b2998947a 0x563b298ac2ed 0x563b2999de1d 0x563b2991fe99 0x563b2991a9ee 0x563b298adbda 0x563b2991fd00 0x563b2991a9ee 0x563b298adbda 0x563b2991c737 0x563b2999ec66 0x563b2991bdaf 0x563b2999ec66 0x563b2991bdaf 0x563b2999ec66 0x563b2991bdaf 0x563b298ae039 0x563b298f1409 0x563b298acc52 0x563b2991fc25 0x563b2991a9ee 0x563b298adbda 0x563b2991c737 0x563b2991a9ee 0x563b298adbda 0x563b2991b915 0x563b298adafa 0x563b2991bc0d 0x563b2991a9ee
     |████████████████████████████████| 881.9 MB 13 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.10.0 which is incompatible.
torchtext 0.10.0 re

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
train_rc = pd.read_csv('/content/drive/MyDrive/Tesi/train_rc.csv')
test_rc = pd.read_csv('/content/drive/MyDrive/Tesi/test_rc.csv')

In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 569072075 bytes (543 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
 59% 4096 Open              --
Path = /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
Type = zip
Physical Size = 569072075

  0%      1% 86 - dataset_bandPass2/192_2b2_Al_mc_LittC2SE_0_bandPass2.wav                                                                    2% 163 - dataset_bandPass2/172_1b4_Ll_mc_AKGC417L_4_bandPass2.wav

In [ ]:
print(train_rc.shape)
print(test_rc.shape)

(5518, 2)
(1380, 2)


In [ ]:
print(train_rc['label'].unique())

['none' 'wheeze' 'crackle' 'both']


In [ ]:
train_4 = train_rc.copy()
train_4 = train_4.replace('none', 0)
train_4 = train_4.replace('crackle', 1)
train_4 = train_4.replace('wheeze', 2)
train_4 = train_4.replace('both',3)
train_4['filename']="/content/dataset_bandPass2/"+train_4['filename']+'_bandPass2.wav'
train_4

,filename,label
0,/content/dataset_bandPass2/194_1b1_Pr_sc_Medit...,0
1,/content/dataset_bandPass2/113_1b1_Ar_sc_Litt3...,0
2,/content/dataset_bandPass2/211_1p2_Pr_mc_AKGC4...,0
3,/content/dataset_bandPass2/120_1b1_Lr_sc_Medit...,0
4,/content/dataset_bandPass2/203_1p4_Pr_mc_AKGC4...,0
...,...,...
5513,/content/dataset_bandPass2/158_1p4_Tc_mc_AKGC4...,0
5514,/content/dataset_bandPass2/154_1b3_Ll_mc_AKGC4...,1
5515,/content/dataset_bandPass2/113_1b1_Lr_sc_Litt3...,0
5516,/content/dataset_bandPass2/204_2b5_Al_mc_AKGC4...,2


In [ ]:
test_4 = test_rc.copy()
test_4 = test_4.replace('none', 0)
test_4 = test_4.replace('crackle', 1)
test_4 = test_4.replace('wheeze', 2)
test_4 = test_4.replace('both',3)
test_4['filename']="/content/dataset_bandPass2/"+test_4['filename']+'_bandPass2.wav'
test_4

,filename,label
0,/content/dataset_bandPass2/157_1b1_Pr_sc_Medit...,1
1,/content/dataset_bandPass2/130_2b2_Al_mc_AKGC4...,1
2,/content/dataset_bandPass2/210_1b1_Ar_sc_Medit...,0
3,/content/dataset_bandPass2/163_8b3_Ll_mc_AKGC4...,0
4,/content/dataset_bandPass2/178_1b6_Pl_mc_AKGC4...,0
...,...,...
1375,/content/dataset_bandPass2/147_1b2_Tc_mc_AKGC4...,0
1376,/content/dataset_bandPass2/192_2b2_Al_mc_LittC...,3
1377,/content/dataset_bandPass2/130_1p2_Pl_mc_AKGC4...,1
1378,/content/dataset_bandPass2/200_2p3_Pr_mc_AKGC4...,0


In [ ]:
train_4.groupby('label').count()

,filename
label,
0,2946
1,1469
2,704
3,399


In [ ]:
test_4.groupby('label').count()

,filename
label,
0,696
1,395
2,182
3,107


In [ ]:
!pip -q install torchaudio

In [ ]:
import torchaudio
duration = 4
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result

In [ ]:
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    )
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 3.3 MB 32.5 MB/s 
     |████████████████████████████████| 895 kB 30.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoModel
DEVICE = 'cuda'
m = AutoModel.from_pretrained("ntu-spml/distilhubert")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for aalbert classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.5)
        self.out_proj = nn.Linear(512, 4)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = features[0]
        res = torch.mean(features, dim=1)
        logit = self.classifier(res)
        return logit

In [ ]:
model = Classifier(m)

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime


def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+2}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/DistilHubert/final_80-20_bandPass2_4-class_distilhubert_base_lr={lr}_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

----------------------------

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=16
duration=4
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Modello finale

In [ ]:
from transformers import AutoModel
DEVICE = 'cuda'
m = AutoModel.from_pretrained("ntu-spml/distilhubert")# torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

In [ ]:
model = Classifier(m)

In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 4secondi tanh dropout=0.5 batch=8 distilhubert final

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loss: 0.99 #nan->0: 100%|██████████| 689/689 [06:22<00:00,  1.80it/s]


loss: 1.118885
acc: 0.527032
start validation


Loss: 0.99 #nan->0: 100%|██████████| 172/172 [01:16<00:00,  2.23it/s]


validation loss: 1.081973671913147
validation accuracy: 0.5581395348837209
Save new model!- 7/11 h17:54
--------------------
* Epoch 2/100


...

validation loss: 1.0160284042358398
validation accuracy: 0.6686046511627907
--------------------
* Epoch 15/100


Loss: 0.18 #nan->0: 100%|██████████| 689/689 [06:21<00:00,  1.80it/s]


loss: 0.307353
acc: 0.895138
start validation


Loss: 1.12 #nan->0: 100%|██████████| 172/172 [01:17<00:00,  2.23it/s]

validation loss: 1.0506528615951538
validation accuracy: 0.6875
--------------------
